In [0]:
%pip install transformers python-doctr==0.10.0 deepdoctection[pt]
%restart_python

In [0]:
import deepdoctection as dd

def process_document(document_path):
    doc_data_return = []
    config_overwrite = ["USE_TABLE_SEGMENTATION=True", "USE_TABLE_REFINEMENT=True"]
    analyzer = dd.get_dd_analyzer(config_overwrite=config_overwrite)
    df = analyzer.analyze(path = document_path)
    df.reset_state()
    doc_iter = iter(df)
    page_num = 0
    while True:
        try:
            page = next(doc_iter)
            page_num += 1
            page_text = page.text
            tables_html = [table.html for table in page.tables]
            
            doc_data_return.append({
                'doc_id': document_path,
                'doc_name': document_path.split('/')[-1],
                'page_num': page_num,
                'page_text': page.text,
                'tables_html': tables_html,
            })
        except StopIteration:
            print(f"last page processed {page_num}")
            break


    return doc_data_return

In [0]:
import os

file_path = "/Volumes/pdf_parsing/test_parsing/raw_files/aph_example_1.pdf"
data_returned = process_document(file_path)

In [0]:
data_returned

In [0]:
volume_path = "/Volumes/pdf_parsing/test_parsing/raw_files"
os.listdir(volume_path)


In [0]:
import os

volume_path = "/Volumes/pdf_parsing/test_parsing/raw_files"
all_doc_data = []

for filename in os.listdir(volume_path):
    file_path = os.path.join(volume_path, filename)
    doc_data = process_document(file_path)
    all_doc_data.extend(doc_data)

In [0]:
df = spark.createDataFrame(all_doc_data)
df.write.mode("overwrite").saveAsTable("pdf_parsing.test_parsing.doc_parsed_data")

In [0]:
%sql
select *
from pdf_parsing.test_parsing.doc_parsed_data

In [0]:
%sql
select *, size(tables_html)
from pdf_parsing.test_parsing.doc_parsed_data

In [0]:
from pyspark.sql.functions import concat_ws, col

df = spark.table("pdf_parsing.test_parsing.doc_parsed_data")
df = df.withColumn("data_to_extract", concat_ws(" ", col("page_text"), col("tables_html")))
df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("pdf_parsing.test_parsing.doc_parsed_data")